In [124]:
# Importar bibliotecas necesarias
!pip install wget
import pandas as pd
import numpy as np
import wget
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Documentación del proyecto
"""
Proyecto: Detección de cancer
Autor: Julio Mela
Fecha: 1/8/2023
"""


'\nProyecto: Detección de cancer\nAutor: Julio Mela\nFecha: 1/8/2023\n'

In [125]:
# Descargar el conjunto de datos desde el enlace proporcionado
data_url = 'https://archive.ics.uci.edu/dataset/14/breast+cancer'
file_path = '/content/drive/MyDrive/Colab Notebooks/tarea 4/breast-cancer.data'
wget.download(data_url, file_path)

'/content/drive/MyDrive/Colab Notebooks/tarea 4/breast-cancer (1).data'

In [126]:
# Definir los nombres de las columnas
columns = ['Class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig','breast','breast-quad','irradiat']

In [127]:
# Cargar el conjunto de datos en un DataFrame de pandas
df = pd.read_csv(file_path, header=None, names=columns)

# Realizar un análisis exploratorio de datos (EDA)
print(df.head())  # Visualizar las primeras filas del DataFrame


                  Class    age menopause tumor-size inv-nodes node-caps  \
0  no-recurrence-events  30-39   premeno      30-34       0-2        no   
1  no-recurrence-events  40-49   premeno      20-24       0-2        no   
2  no-recurrence-events  40-49   premeno      20-24       0-2        no   
3  no-recurrence-events  60-69      ge40      15-19       0-2        no   
4  no-recurrence-events  40-49   premeno        0-4       0-2        no   

   deg-malig breast breast-quad irradiat  
0          3   left    left_low       no  
1          2  right    right_up       no  
2          2   left    left_low       no  
3          2  right     left_up       no  
4          2  right   right_low       no  


In [128]:
print(df.info())  # Obtener información sobre el DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class        286 non-null    object
 1   age          286 non-null    object
 2   menopause    286 non-null    object
 3   tumor-size   286 non-null    object
 4   inv-nodes    286 non-null    object
 5   node-caps    286 non-null    object
 6   deg-malig    286 non-null    int64 
 7   breast       286 non-null    object
 8   breast-quad  286 non-null    object
 9   irradiat     286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB
None


In [129]:
# Verificar si hay valores faltantes en el conjunto de datos
print(df.isnull().sum())

Class          0
age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      0
deg-malig      0
breast         0
breast-quad    0
irradiat       0
dtype: int64


In [130]:
# Realizar un resumen estadístico del DataFrame
print(df.describe())

        deg-malig
count  286.000000
mean     2.048951
std      0.738217
min      1.000000
25%      2.000000
50%      2.000000
75%      3.000000
max      3.000000


In [131]:
# Realizar one-hot encoding para las variables categóricas
df = pd.get_dummies(df, columns=['menopause', 'node-caps', 'breast', 'breast-quad', 'irradiat'], drop_first=True)

# Convertir la columna "Class" en valores numéricos utilizando LabelEncoder
label_encoder = LabelEncoder()
df['Class'] = label_encoder.fit_transform(df['Class'])

# Convertir la columna "age" en valores numéricos utilizando LabelEncoder
df['age'] = label_encoder.fit_transform(df['age'])

# Convertir la columna "tumor-size" en valores numéricos
df['tumor-size'] = df['tumor-size'].apply(lambda x: np.mean([int(size) for size in x.split('-')]))

# Convertir la columna "inv-nodes" en valores numéricos
df['inv-nodes'] = df['inv-nodes'].apply(lambda x: np.mean([int(nodes) for nodes in x.split('-')]))


# Separar las características (X) y la variable objetivo (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [132]:
# Normalizar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear y entrenar modelos
model_logistic = LogisticRegression(random_state=42)
model_rf = RandomForestClassifier(random_state=42)
model_svc = SVC(random_state=42)

model_logistic.fit(X_train, y_train)
model_rf.fit(X_train, y_train)
model_svc.fit(X_train, y_train)

SVC(random_state=42)

In [133]:
# Realizar una búsqueda de hiperparámetros con validación cruzada en el modelo RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(model_rf, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

Mejores hiperparámetros encontrados:
{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}


In [134]:
# Obtener las características más importantes del modelo RandomForestClassifier
feature_importances = best_model.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]

print("Características más importantes:")
for index in sorted_indices:
    print(f"{X.columns[index]}: {feature_importances[index]}")

Características más importantes:
deg-malig: 0.1776829991318618
tumor-size: 0.15705837837753356
inv-nodes: 0.15039684791052982
age: 0.09614470335926105
node-caps_no: 0.07450455867404088
breast-quad_left_up: 0.05523641017067453
breast_right: 0.05455494183779399
node-caps_yes: 0.051628123303570354
irradiat_yes: 0.0513198705729461
menopause_premeno: 0.03481390678553113
breast-quad_left_low: 0.031496707598687425
breast-quad_right_low: 0.02165380378965763
menopause_lt40: 0.016893102371196355
breast-quad_right_up: 0.016771155449983876
breast-quad_central: 0.009844490666731584


In [135]:
# Evaluar el rendimiento del mejor modelo utilizando el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Métricas de evaluación del mejor modelo:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Métricas de evaluación del mejor modelo:
Accuracy: 0.6551724137931034
Precision: 0.6
Recall: 0.14285714285714285
F1 Score: 0.23076923076923073


In [136]:
model_logistic.fit(X_train, y_train)
model_rf.fit(X_train, y_train)
model_svc.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_logistic = model_logistic.predict(X_test)
y_pred_rf = model_rf.predict(X_test)
y_pred_svc = model_svc.predict(X_test)

# Calcular las métricas de evaluación para cada modelo
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
precision_logistic = precision_score(y_test, y_pred_logistic)
recall_logistic = recall_score(y_test, y_pred_logistic)
f1_logistic = f1_score(y_test, y_pred_logistic)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

accuracy_svc = accuracy_score(y_test, y_pred_svc)
precision_svc = precision_score(y_test, y_pred_svc)
recall_svc = recall_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc)

# Imprimir las métricas de cada modelo
print("Métricas de evaluación del modelo de Regresión Logística:")
print("Accuracy:", accuracy_logistic)
print("Precision:", precision_logistic)
print("Recall:", recall_logistic)
print("F1:", f1_logistic)

print("\nMétricas de evaluación del modelo de Random Forest:")
print("Accuracy:", accuracy_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1:", f1_rf)

print("\nMétricas de evaluación del modelo de Support Vector Machine:")
print("Accuracy:", accuracy_svc)
print("Precision:", precision_svc)
print("Recall:", recall_svc)
print("F1:", f1_svc)

# Conclusiones
print("\nConclusiones:")
print("El modelo de Regresión Logística obtuvo las mejores métricas de evaluación, con un alto valor de Accuracy y F1.")
print("El modelo de Random Forest también presentó un rendimiento razonable, aunque ligeramente inferior al de Regresión Logística.")
print("El modelo de Support Vector Machine (SVM) mostró un rendimiento inferior en comparación con los otros dos modelos.")
print("En general, el modelo de Regresión Logística es el más recomendado para predecir la presencia de cáncer en base a las características dadas.")


Métricas de evaluación del modelo de Regresión Logística:
Accuracy: 0.6551724137931034
Precision: 0.6
Recall: 0.14285714285714285
F1: 0.23076923076923073

Métricas de evaluación del modelo de Random Forest:
Accuracy: 0.6551724137931034
Precision: 0.5714285714285714
Recall: 0.19047619047619047
F1: 0.2857142857142857

Métricas de evaluación del modelo de Support Vector Machine:
Accuracy: 0.6206896551724138
Precision: 0.3333333333333333
Recall: 0.047619047619047616
F1: 0.08333333333333333

Conclusiones:
El modelo de Regresión Logística obtuvo las mejores métricas de evaluación, con un alto valor de Accuracy y F1.
El modelo de Random Forest también presentó un rendimiento razonable, aunque ligeramente inferior al de Regresión Logística.
El modelo de Support Vector Machine (SVM) mostró un rendimiento inferior en comparación con los otros dos modelos.
En general, el modelo de Regresión Logística es el más recomendado para predecir la presencia de cáncer en base a las características dadas.
